
# Importação Bibliotecas

In [1]:
!pip install pyarrow
!pip install fastparquet
!pip install pandas
!pip install numpy
!pip install seaborn
!pip install matplotlib
!pip install scikit-learn
!pip install scipy.stats
!pip install dask dask[dataframe]
!pip install tslearn
!pip install tensorflow

ERROR: Could not find a version that satisfies the requirement scipy.stats (from versions: none)
ERROR: No matching distribution found for scipy.stats


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/ba/7c/b971f2485155917ecdcebb210e021e36a6b65457394590be01cc61515310/tensorflow-2.13.0-cp310-cp310-win_amd64.whl.metadata
  Using cached tensorflow-2.13.0-cp310-cp310-win_amd64.whl.metadata (2.6 kB)
  Obtaining dependency information for tensorflow-intel==2.13.0 from https://files.pythonhosted.org/packages/40/fa/98115f6fe4d92e1962f549917be2dc8e369853b7e404191996fedaaf4dd6/tensorflow_intel-2.13.0-cp310-cp310-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.13.0-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Obtaining dependency information for h5py>=2.9.0 from https://files.pythonhosted.org/packages/e2/c4/6f8dae1530d57a6122fd5b72c750187484acbe612f630cb2179e4bcb12c1/

ERROR: Could not install packages due to an OSError: [WinError 2] O sistema não pode encontrar o arquivo especificado: 'C:\\Python310\\Scripts\\pyrsa-decrypt.exe' -> 'C:\\Python310\\Scripts\\pyrsa-decrypt.exe.deleteme'



In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import scipy.stats as stats
import os
import dask.dataframe as dd

# Extração dataset

In [4]:
def concatenate_and_count_rows_in_parquet_files(file_path, filtered_cols):
    total_rows = 0
    list_of_dfs = []  # Lista para armazenar os DataFrames lidos
    
    # Lista de arquivos Parquet no diretório especificado
    parquet_files = [file for file in os.listdir(file_path) if file.endswith('.parquet')]
    
    # Loop para ler cada arquivo Parquet, contar as linhas e armazenar o DataFrame na lista
    for file_name in parquet_files:
        full_path = os.path.join(file_path, file_name)
        df = dd.read_parquet(full_path, columns=filtered_cols)
        total_rows += len(df)
        list_of_dfs.append(df)
        print(f"Arquivo {file_name} tem {len(df)} linhas.")
    
    # Concatenar todos os Dask DataFrames em um único Dask DataFrame
    concatenated_df = dd.concat(list_of_dfs, interleave_partitions=True)
    
    return concatenated_df, total_rows

# Uso da função
file_path = "primeiro_arquivo_zip/"
filtered_cols = ['recording_time', 'dateDay-1', 'dateMonth-1', 'dateYear-1', 'phaseOfFlight-1',
                 'message0418DAA-1','message0422DAA-1','amscHprsovDrivF-1a', 'amscHprsovDrivF-1b',
                 'amscHprsovDrivF-2b', 'amscPrsovDrivF-1a',
                 'amscPrsovDrivF-1b', 'amscPrsovDrivF-2b',
                 'basBleedLowPressF-1a', 'basBleedLowPressF-2b',
                 'basBleedLowTempF-1a', 'basBleedLowTempF-2b',
                 'basBleedOverPressF-1a', 'basBleedOverPressF-2b',
                 'basBleedOverTempF-1a', 'basBleedOverTempF-2b',
                 'bleedFavTmCmd-1a', 'bleedFavTmCmd-1b',
                 'bleedFavTmCmd-2a', 'bleedFavTmCmd-2b', 'bleedFavTmFbk-1a',
                 'bleedFavTmFbk-1b', 'bleedFavTmFbk-2b', 'bleedHprsovCmdStatus-1a',
                 'bleedHprsovCmdStatus-1b', 'bleedHprsovCmdStatus-2a',
                 'bleedHprsovCmdStatus-2b', 'bleedHprsovOpPosStatus-1a',
                 'bleedHprsovOpPosStatus-1b', 'bleedHprsovOpPosStatus-2a',
                 'bleedHprsovOpPosStatus-2b', 'bleedMonPress-1a',
                 'bleedMonPress-1b', 'bleedMonPress-2a', 'bleedMonPress-2b',
                 'bleedOnStatus-1a', 'bleedOnStatus-1b', 'bleedOnStatus-2b',
                 'bleedOverpressCas-2a', 'bleedOverpressCas-2b',
                 'bleedPrecoolDiffPress-1a', 'bleedPrecoolDiffPress-1b',
                 'bleedPrecoolDiffPress-2a', 'bleedPrecoolDiffPress-2b',
                 'bleedPrsovClPosStatus-1a', 'bleedPrsovClPosStatus-2a',
                 'bleedPrsovFbk-1a']

resulting_df, total_rows = concatenate_and_count_rows_in_parquet_files(file_path, filtered_cols)
print(f"O número total de linhas em todos os arquivos é: {total_rows}")

Arquivo TCRF_ARCHIVE_06120089_20221212212501.parquet tem 6002 linhas.
Arquivo TCRF_ARCHIVE_06120089_20221212212502.parquet tem 6081 linhas.
Arquivo TCRF_ARCHIVE_06120089_20221212212504.parquet tem 10301 linhas.
Arquivo TCRF_ARCHIVE_06120089_20221212212505.parquet tem 41361 linhas.
Arquivo TCRF_ARCHIVE_06120089_20221212212506.parquet tem 6961 linhas.
Arquivo TCRF_ARCHIVE_06120089_20221212212507.parquet tem 17721 linhas.
Arquivo TCRF_ARCHIVE_06120089_20221212212508.parquet tem 45721 linhas.
Arquivo TCRF_ARCHIVE_06120089_20221212212509.parquet tem 21781 linhas.
Arquivo TCRF_ARCHIVE_06120089_20221212212510.parquet tem 46661 linhas.
Arquivo TCRF_ARCHIVE_06120089_20221212212511.parquet tem 28741 linhas.
Arquivo TCRF_ARCHIVE_06120089_20221212212512.parquet tem 13981 linhas.
Arquivo TCRF_ARCHIVE_06120089_20221212212513.parquet tem 60010 linhas.
Arquivo TCRF_ARCHIVE_06120089_20221212212514.parquet tem 7741 linhas.
Arquivo TCRF_ARCHIVE_06120089_20221212212515.parquet tem 8081 linhas.
Arquivo TCR

In [ ]:
resulting_df.info()

In [ ]:
resulting_df.describe()

# Pré processamento dos dados

In [5]:
def remove_duplicates_except_record_time(dask_df):
    """
    Remove linhas duplicadas com base em todas as colunas, exceto 'record_time'.

    Parâmetros:
    - dask_df: DataFrame Dask de entrada.

    Retorna:
    - DataFrame Dask com linhas duplicadas removidas.
    """
    
    # Crie uma lista de colunas a serem consideradas ao identificar duplicatas
    columns_to_consider = [col for col in dask_df.columns if col != 'record_time']
    
    # Use o método drop_duplicates do Dask para remover duplicatas com base nas colunas especificadas
    #Mantém o primeiro valor duplicado por default
    unique_df = dask_df.drop_duplicates(subset=columns_to_consider)
    
    return unique_df

# Uso da função
unique_df = remove_duplicates_except_record_time(resulting_df)


In [25]:
# Contar o número de linhas em 'unique_df' (depois da remoção de duplicatas)
after_row_count = len(unique_df)

# Imprimir os resultado
print(f"Número de linhas depois da remoção de duplicatas: {after_row_count}")

Número de linhas depois da remoção de duplicatas: 1711622


### Rotulagem de dados

In [ ]:
# Pegando uma amostra de 5% do conjunto de dados
sample_df = unique_df.sample(frac=0.05).compute()

# Verificando valores únicos na amostra para 'message0418DAA-1'
unique_values_0418_sample = sample_df['message0418DAA-1'].unique()
print("Valores únicos na amostra para 'message0418DAA-1':", unique_values_0418_sample)

# Verificando valores únicos na amostra para 'message0422DAA-1'
unique_values_0422_sample = sample_df['message0422DAA-1'].unique()
print("Valores únicos na amostra para 'message0422DAA-1':", unique_values_0422_sample)

In [6]:
# 1. Criar Coluna de Rótulo
def label_rows(row):
    if (row['message0418DAA-1'] > 0) or (row['message0422DAA-1'] > 0):
        return 'falha'
    else:
        return 'normal'

unique_df = unique_df.map_partitions(lambda df: df.assign(label=df.apply(label_rows, axis=1)))

In [7]:
# 2. Verificar a presença de NaNs antes da transformação
nan_counts_before = unique_df[['dateYear-1', 'dateMonth-1', 'dateDay-1']].isna().sum().compute()
print("NaN counts before transformation:")
print(nan_counts_before)

# 3. Tratar os NaNs
unique_df = unique_df.fillna({'dateYear-1': 0, 'dateMonth-1': 0, 'dateDay-1': 0})

# 4. Função para criar a coluna 'full_date'
def create_full_date(row):
    year = str(int(row['dateYear-1']))
    month = str(int(row['dateMonth-1'])).zfill(2)  # Garante que o mês tenha 2 dígitos
    day = str(int(row['dateDay-1'])).zfill(2)  # Garante que o dia tenha 2 dígitos
    return f"{year}-{month}-{day}"

unique_df = unique_df.map_partitions(lambda df: df.assign(full_date=df.apply(create_full_date, axis=1)))

# 5. Verificar a presença de NaNs após a transformação
nan_counts_after = unique_df[['dateYear-1', 'dateMonth-1', 'dateDay-1']].isna().sum().compute()
print("\nNaN counts after transformation:")
print(nan_counts_after)

NaN counts before transformation:
dateYear-1     1627737
dateMonth-1    1627737
dateDay-1      1627737
dtype: int64

NaN counts after transformation:
dateYear-1     0
dateMonth-1    0
dateDay-1      0
dtype: int64


### Extração de características

#### Regressão logística

In [8]:
# Lista de colunas que são fluxos de sensor
sensor_columns = [col for col in unique_df.columns if col not in ['label', 'full_date']]

# Calcular estatísticas básicas
stats_df = unique_df[sensor_columns].describe().compute()

# Se você quiser adicionar mais características, pode fazê-lo aqui.

In [9]:
# Convertendo o DataFrame Dask para Pandas
unique_df_pd = unique_df.compute()

X = unique_df_pd.drop(columns=['label', 'full_date'])
y = unique_df_pd['label']

# Dividindo os dados
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [13]:
from sklearn.impute import SimpleImputer

# Criar um imputador que substitui NaN por 0
imputer = SimpleImputer(strategy='constant', fill_value=0)

# Ajustar o imputador aos dados de treinamento e transformá-los
X_train_imputed = imputer.fit_transform(X_train)

# Transformar os dados de teste usando o mesmo imputador
X_test_imputed = imputer.transform(X_test)

# Agora, treine o modelo LogisticRegression nos dados imputados
clf.fit(X_train_imputed, y_train)

LogisticRegression(max_iter=10000)

In [14]:
from sklearn.metrics import accuracy_score

# Fazer previsões no conjunto de teste
y_pred = clf.predict(X_test_imputed)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia no conjunto de teste: {accuracy:.4f}")

Acurácia no conjunto de teste: 0.9996


In [17]:
print(unique_df.columns)

Index(['recording_time', 'dateDay-1', 'dateMonth-1', 'dateYear-1',
       'phaseOfFlight-1', 'message0418DAA-1', 'message0422DAA-1',
       'amscHprsovDrivF-1a', 'amscHprsovDrivF-1b', 'amscHprsovDrivF-2b',
       'amscPrsovDrivF-1a', 'amscPrsovDrivF-1b', 'amscPrsovDrivF-2b',
       'basBleedLowPressF-1a', 'basBleedLowPressF-2b', 'basBleedLowTempF-1a',
       'basBleedLowTempF-2b', 'basBleedOverPressF-1a', 'basBleedOverPressF-2b',
       'basBleedOverTempF-1a', 'basBleedOverTempF-2b', 'bleedFavTmCmd-1a',
       'bleedFavTmCmd-1b', 'bleedFavTmCmd-2a', 'bleedFavTmCmd-2b',
       'bleedFavTmFbk-1a', 'bleedFavTmFbk-1b', 'bleedFavTmFbk-2b',
       'bleedHprsovCmdStatus-1a', 'bleedHprsovCmdStatus-1b',
       'bleedHprsovCmdStatus-2a', 'bleedHprsovCmdStatus-2b',
       'bleedHprsovOpPosStatus-1a', 'bleedHprsovOpPosStatus-1b',
       'bleedHprsovOpPosStatus-2a', 'bleedHprsovOpPosStatus-2b',
       'bleedMonPress-1a', 'bleedMonPress-1b', 'bleedMonPress-2a',
       'bleedMonPress-2b', 'bleedOnSta

In [19]:
# Lista de colunas para calcular as características
feature_columns = [
    'amscHprsovDrivF-1a', 'amscHprsovDrivF-1b', 'amscHprsovDrivF-2b',
    'amscPrsovDrivF-1a', 'amscPrsovDrivF-1b', 'amscPrsovDrivF-2b',
    'basBleedLowPressF-1a', 'basBleedLowPressF-2b', 'basBleedLowTempF-1a',
    'basBleedLowTempF-2b', 'basBleedOverPressF-1a', 'basBleedOverPressF-2b',
    'basBleedOverTempF-1a', 'basBleedOverTempF-2b', 'bleedFavTmCmd-1a',
    'bleedFavTmCmd-1b', 'bleedFavTmCmd-2a', 'bleedFavTmCmd-2b',
    'bleedFavTmFbk-1a', 'bleedFavTmFbk-1b', 'bleedFavTmFbk-2b',
    'bleedHprsovCmdStatus-1a', 'bleedHprsovCmdStatus-1b',
    'bleedHprsovCmdStatus-2a', 'bleedHprsovCmdStatus-2b',
    'bleedHprsovOpPosStatus-1a', 'bleedHprsovOpPosStatus-1b',
    'bleedHprsovOpPosStatus-2a', 'bleedHprsovOpPosStatus-2b',
    'bleedMonPress-1a', 'bleedMonPress-1b', 'bleedMonPress-2a',
    'bleedMonPress-2b', 'bleedOnStatus-1a', 'bleedOnStatus-1b',
    'bleedOnStatus-2b', 'bleedOverpressCas-2a', 'bleedOverpressCas-2b',
    'bleedPrecoolDiffPress-1a', 'bleedPrecoolDiffPress-1b',
    'bleedPrecoolDiffPress-2a', 'bleedPrecoolDiffPress-2b',
    'bleedPrsovClPosStatus-1a', 'bleedPrsovClPosStatus-2a',
    'bleedPrsovFbk-1a'
]

# Calcular média, mediana e desvio padrão para as colunas listadas
mean_features = unique_df[feature_columns].mean().compute()
median_features = unique_df[feature_columns].median().compute()
std_features = unique_df[feature_columns].std().compute()

# Criar um DataFrame com os resultados
features_df = pd.DataFrame({
    'mean': mean_features,
    'median': median_features,
    'std': std_features
})

print(features_df)

                                mean   median        std
amscHprsovDrivF-1a          0.000000   0.0000   0.000000
amscHprsovDrivF-1b          0.000000   0.0000   0.000000
amscHprsovDrivF-2b          0.000012   0.0000   0.003453
amscPrsovDrivF-1a           0.000000   0.0000   0.000000
amscPrsovDrivF-1b           0.000000   0.0000   0.000000
amscPrsovDrivF-2b           0.000000   0.0000   0.000000
basBleedLowPressF-1a        0.000000   0.0000   0.000000
basBleedLowPressF-2b        0.000000   0.0000   0.000000
basBleedLowTempF-1a         0.000000   0.0000   0.000000
basBleedLowTempF-2b         0.000000   0.0000   0.000000
basBleedOverPressF-1a       0.000000   0.0000   0.000000
basBleedOverPressF-2b       0.000000   0.0000   0.000000
basBleedOverTempF-1a        0.000000   0.0000   0.000000
basBleedOverTempF-2b        0.003573   0.0000   0.059669
bleedFavTmCmd-1a           31.799998   0.0000  53.046006
bleedFavTmCmd-1b           65.930241  95.0000  56.857361
bleedFavTmCmd-2a           28.5

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

# Imputar os valores NaN
imputer = SimpleImputer(strategy='constant', fill_value=0)
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Treinar o modelo
clf = LogisticRegression(max_iter=10000)
clf.fit(X_train_imputed, y_train)

# Obter a importância das características
importance = clf.coef_[0]

# Mapear a importância para os nomes das colunas
feature_importance = dict(zip(X.columns, importance))

# Agora, você pode filtrar as características com base em um limiar de importância
importance_threshold = 0.05  # Ajuste conforme necessário
important_features = [feature for feature, importance in feature_importance.items() if abs(importance) > importance_threshold]

# Filtrar o DataFrame para manter apenas as características importantes
filtered_df = unique_df[important_features]

In [23]:
filtered_df.describe()

,bleedFavTmCmd-2a,bleedFavTmCmd-2b,bleedFavTmFbk-2b,bleedMonPress-1a,bleedMonPress-1b,bleedMonPress-2a,bleedMonPress-2b,bleedPrsovFbk-1a
npartitions=1,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...


#### Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Inicializar o classificador Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Treinar o modelo usando os dados de treinamento imputados
rf_clf.fit(X_train_imputed, y_train)

# Fazer previsões no conjunto de teste
y_pred_rf = rf_clf.predict(X_test_imputed)

# Avaliar o desempenho do modelo
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Acurácia do Random Forest no conjunto de teste: {accuracy_rf:.4f}")

# Mostrar relatório de classificação
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_rf))

Acurácia do Random Forest no conjunto de teste: 1.0000

Relatório de Classificação:
              precision    recall  f1-score   support

       falha       1.00      1.00      1.00       144
      normal       1.00      1.00      1.00    342181

    accuracy                           1.00    342325
   macro avg       1.00      1.00      1.00    342325
weighted avg       1.00      1.00      1.00    342325

